## Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-licence]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,135 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../s

## Installing Libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F  ## Contains Activation Function
import torch.autograd as autograd  # For automatic Computing Gradient
from torch.autograd import Variable # For doing Automatic Differentiation
from collections import deque, namedtuple # Double Ended Queue , Tuple

## Part - 1 Building AI

## Creating the Architecture of the Neural Network

In [ ]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):
    super(Network,self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size , 64) # fc1 = first fully connected layer
    self.fc2 = nn.Linear(64,64)  # Second fully connected layer
    self.fc3 = nn.Linear(64,action_size)  # Third fully connected layer

  def forward(self,state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

## Part - 2 Training AI

## Setting up the Environment

In [ ]:
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size:', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size: 8
Number of actions:  4


## Initializing the Hyperparameters

In [ ]:
learning_rate = 5e-4 # (It will learn itself)
minibatch_size = 64 # (Determines how many samples can be processed at a time)
discount_factor = 0.99 # gamma (Nakki karse reward apvo ke nai)
reply_buffer_size = int(1e5)  # (Capacity) -> (it will store experiences like action,state,reward)
interpolation_parameter = 1e-3 ## Function Approximation karse aa

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Implementing Experience Replay

In [ ]:
class ReplyMemory(object):

  def __init__(self,capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity ## Maximum number of Experineces that memory can store
    self.memory = [] ## Initialize an empty list for storing the Experience

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity: ## Must ::>> len(Capacity) > len(Memory)
      del self.memory[0]

  def sample(self,batch_size):
    if len(self.memory) < batch_size: ## Must ::>> memory(size) > batch_size
      return None
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.tensor(np.vstack([e[4] for e in experiences if e is not None]), dtype=torch.uint8).to(self.device) ## It will use 8 bit per element copare to np.floate 32 or np.float64
    return states, actions, rewards, next_states, dones

## Implementing DQN class

In [ ]:
class Agent():

  def __init__(self,state_size,action_size):
    self.device = torch.device("cudo:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size,action_size).to(self.device)  # Represents local Q_network which will be updates frequently during training and it will predict the Q-values
    self.target_qnetwork = Network(state_size,action_size).to(self.device) # represents Target Q-network which will less updates and helps to stabilize the learning process
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)  # optimize the parameters
    self.memory = ReplyMemory(reply_buffer_size) # reply_buffer_size defines maximum capacity of the reply buffer
    self.t_step = 0 # (Counter will counts at which moment we to learn)

  def step(self, state, next_state, action, reward, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_steps = (self.t_step + 1) % 4
    if self.t_step == 0 :
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(minibatch_size)
        self.learn(experiences, discount_factor) ## (Gamma = discount_factor)

  def act(self,state, epsilon = 0.):
      state = torch.from_numpy(state).float().unsqueeze(0).to(self.device) ## unsqueeze adds a batch dimensions to the tensor making it for processing a single state
      self.local_qnetwork.eval()  ## states the Q-network to the evaluation mode
      with torch.no_grad(): # Any gradient computation will be diabled (improves eficiency)
        action_values = self.local_qnetwork(state)
      self.local_qnetwork.train()
      if random.random() > epsilon:
        return np.argmax(action_values.cpu().data.numpy())  # Select action with the higest Q-value
      else:
        return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, actions, rewards, next_states, dones = experiences
    next_q_target = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_target = rewards + (discount_factor * next_q_target * (1-dones))
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_target)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

## Initialiazing  DQN agent

In [ ]:
agent = Agent(state_size, number_actions)

## Training the DQN agent

In [ ]:
number_episodes = 100
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episodes in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _  = env.step(action)
    agent.step(state, next_state, action, reward, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episodes,np.mean(scores_on_100_episodes)), end = "" )
  if episodes % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episodes, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 300.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episodes - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -101.15
Episode 200	Average Score: -39.96
Episode 300	Average Score: 28.31
Episode 400	Average Score: 92.85
Episode 500	Average Score: 230.66
Episode 600	Average Score: 270.91
Episode 700	Average Score: 259.82
Episode 800	Average Score: 245.57
Episode 900	Average Score: 269.52
Episode 1000	Average Score: 258.00
Episode 1100	Average Score: 245.42
Episode 1200	Average Score: 268.31
Episode 1300	Average Score: 274.99
Episode 1400	Average Score: 275.53
Episode 1500	Average Score: 269.18
Episode 1600	Average Score: 276.73
Episode 1700	Average Score: 269.77
Episode 1800	Average Score: 274.53
Episode 1900	Average Score: 247.17
Episode 2000	Average Score: 257.19
Episode 2100	Average Score: 257.62
Episode 2200	Average Score: 261.72
Episode 2300	Average Score: 234.01
Episode 2400	Average Score: 254.30
Episode 2500	Average Score: 255.38
Episode 2600	Average Score: 255.56
Episode 2700	Average Score: 253.56
Episode 2800	Average Score: 270.46
Episode 2900	Average Score: 27

## Part - 3 Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
  env = gym.make(env_name, render_mode='rgb_array')
  state, _ = env. reset ()
  done = False
  Frames = []
  while not done:
    Frame = env.render()
    Frames.append(Frame)
    action = agent.act(state)
    state, reward, done, _,_ = env.step(action.item())
  env.close()
  imageio.mimsave('video.mp4', Frames, fps=30)

show_video_of_model(agent,'LunarLander-v2')

def show_video():
  mp4list = glob.glob('*.mp4')
  if len(mp4list)>0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    display(HTML(data='''<video alt="test" autoplay
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")

show_video()